In [1]:
import numpy as np
import pandas as pd

In [2]:
df2003 = pd.read_csv("SoccerFiles/PLGames2003.csv", error_bad_lines = False)

b'Skipping line 305: expected 57 fields, saw 72\nSkipping line 306: expected 57 fields, saw 72\nSkipping line 307: expected 57 fields, saw 72\nSkipping line 308: expected 57 fields, saw 72\nSkipping line 309: expected 57 fields, saw 72\nSkipping line 310: expected 57 fields, saw 72\nSkipping line 311: expected 57 fields, saw 72\nSkipping line 312: expected 57 fields, saw 72\nSkipping line 313: expected 57 fields, saw 72\nSkipping line 314: expected 57 fields, saw 72\nSkipping line 315: expected 57 fields, saw 72\nSkipping line 316: expected 57 fields, saw 72\nSkipping line 317: expected 57 fields, saw 72\nSkipping line 318: expected 57 fields, saw 72\nSkipping line 319: expected 57 fields, saw 72\nSkipping line 320: expected 57 fields, saw 72\nSkipping line 321: expected 57 fields, saw 72\nSkipping line 322: expected 57 fields, saw 72\nSkipping line 323: expected 57 fields, saw 72\nSkipping line 324: expected 57 fields, saw 72\nSkipping line 325: expected 57 fields, saw 72\nSkipping li

In [13]:
#print(df2003.columns)
columns = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 
           'FTR', 'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 
           'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']

df2003 = df2003[columns]

print(df2003.columns)



Index(['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR',
       'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY',
       'HR', 'AR'],
      dtype='object')


In [15]:
years = {}
features = {'HomeGoals': 0, 'AwayGoals': 0, 
            'Wins': 0, 'Losses': 0, 'Draws': 0,
            'HalfTimeHomeGoals': 0, 'HalfTimeAwayGoals': 0,
            'HalfTimeWins' : 0, 'HalfTimeLosses' : 0, 
            'HalfTimeDraws': 0,
            'HalfTimeChoke': 0, # They were winning at halftime but lost the game
            'HalfTimeClutch': 0, # They were losing at halftime but won the game
            # Number of times they were winning but lost
            # 
            'AVGCrowd': 0, 'AVGHomeShots': 0, 'AVGAwayShots': 0,
            'AVGHomeWoodwork': 0, 'AVGAwayWoodwork': 0, 
            'AVGHomeCorners': 0, 'AVGAwayCorners': 0,
            'AVGHomeFouls': 0, 'AVGAwayFouls': 0,
            'AVGHomeFreeKicks': 0, 'AVGAwayFreeKicks': 0,
            'AVGHomeOffsides': 0, 'AVGAwayOffsides': 0,
            'AVGHomeYellow': 0, 'AVGAwayYellow': 0,
            'AVGHomeRed': 0, 'AVGAwayRed': 0,
            'Won15': 0, 'Won25': 0
           }
# I'm not sure how to include Referee

In [14]:
d2003 = dict()
for index, row in df2003.iterrows():
    if row['HomeTeam'] not in d2003:
        d2003[row['HomeTeam']] = dict(features)
    
    if row['AwayTeam'] not in d2003:
        d2003[row['AwayTeam']] = dict(features)
    
    
    # Add Home Team Stats
    d2003[row['HomeTeam']]['HomeGoals'] += row['FTHG']
    d2003[row['HomeTeam']]['HalfTimeHomeGoals'] += row['HTHG']
    
    
    # Add Away Team Stats
    d2003[row['AwayTeam']]['AwayGoals'] = row['FTAG']
    d2003[row['AwayTeam']]['HalfTimeAwayGoals'] += row['HTAG']
    
    
    # Determine Winner
    if row['FTR'] == 'H':
        d2003[row['HomeTeam']]['Wins'] += 1
        d2003[row['AwayTeam']]['Losses'] += 1
    elif row['FTR'] == 'A':
        d2003[row['HomeTeam']]['Losses'] += 1
        d2003[row['AwayTeam']]['Wins'] += 1
    else:
        # Draw
        d2003[row['HomeTeam']]['Draws'] += 1
        d2003[row['AwayTeam']]['Draws'] += 1
        
    # Determine HalfTime Winner
    if row['HTR'] == 'H':
        d2003[row['HomeTeam']]['HalfTimeWins'] += 1
        d2003[row['AwayTeam']]['HalfTimeLosses'] += 1
    elif row['HTR'] == 'A':
        d2003[row['HomeTeam']]['HalfTimeLosses'] += 1
        d2003[row['AwayTeam']]['HalfTimeWins'] += 1
    else:
        # Draw
        d2003[row['HomeTeam']]['HalfTimeDraws'] += 1
        d2003[row['AwayTeam']]['HalfTimeDraws'] += 1
    
    
    if (row['HTR'] == 'H') and (row['FTR'] == 'A'):
        d2003[row['HomeTeam']]['HalfTimeChoke'] += 1
        d2003[row['HomeTeam']]['HalfTimeClutch'] += 1
    elif (row['HTR'] == 'A') and (row['FTR'] == 'H'):
        d2003[row['HomeTeam']]['HalfTimeClutch'] += 1
        d2003[row['HomeTeam']]['HalfTimeChoke'] += 1
        
    for k, v in d2003.items():
        print(k)
        print(v)
        print()
        break
    
    print(row)
    
    break


Arsenal
{'HomeGoals': 3, 'AwayGoals': 0, 'Wins': 1, 'Losses': 0, 'Draws': 0, 'HalfTimeHomeGoals': 0, 'HalfTimeAwayGoals': 0, 'HalfTimeResult': 0, 'HalfTimeLoseButWin': 0, 'HalfTimeWinButLose': 0, 'AVGCrowd': 0, 'AVGHomeShots': 0, 'AVGAwayShots': 0, 'AVGHomeWoodwork': 0, 'AVGAwayWoodwork': 0, 'AVGHomeCorners': 0, 'AVGAwayCorners': 0, 'AVGHomeFouls': 0, 'AVGAwayFouls': 0, 'AVGHomeFreeKicks': 0, 'AVGAwayFreeKicks': 0, 'AVGHomeOffsides': 0, 'AVGAwayOffsides': 0, 'AVGHomeYellow': 0, 'AVGAwayYellow': 0, 'AVGHomeRed': 0, 'AVGAwayRed': 0, 'Won15': 0, 'Won25': 0}

HomeTeam     Arsenal
AwayTeam     Everton
FTHG               2
FTAG               1
FTR                H
HTHG               1
HTAG               0
HTR                H
Referee     M Halsey
HS                11
AS                13
HST                5
AST                7
HF                 8
AF                15
HC                 6
AC                 9
HY                 1
AY                 3
HR                 1
AR                

In [ ]:
# Divide each "average" category by 38